# Why NVIS antenna configuration?
## Properties
* easy to setup : no installation on height needed
* hard to jam and locate (direction finding) : energy goes straight up
* simple and cheap antenna : horizontal dipole
* omnidirectional coverage
* works well with low power
* range limited to a few 100 kms
* frequency limited to 2-10MHz, depending on ionospheric conditions
    * 4 - 8MHz during day : 40m band
    * 2 - 4MHz at night : 80m band
* other party should also have NVIS antenna

## Frequencies
* Must be high enough, otherwise too much D-layer absorption
* Must be low enough to be refracted by F-layer.  Maximum frequency is the Ionospheric foF2 critical frequency.
* [Space Weather Services - Ionospheric Map](https://www.sws.bom.gov.au/HF_Systems/6/5)
* 80m band seems to be the only viable option in western Europe

## Setup
* Radiation angle (typically 75° for NVIS) increases as active element is lowered (typicallyλ 0.10 to 0.25λ above ground)
* Mounting horizontal dipole lower means more ground losses and need for retuning (shortening)

## Construction of a 40m-60m-80m dipole
### Materials
* Wire : 
    * #AWG20 (0.5mm²) to #AWG26 (0.13mm²) used for mobile installation
    * Stranded copper for ease of handling
    * For 40m dipole, buy length of 25m
* 2 insulators : 2 pieces of FR4, with 3 holes spaced about 15mm apart (works probably better than egg-insulator or dog-bone insulator for stranded wire)

## U01 Antenna
* Sloped EFHW
* Ferrite core :
    * [Amidon FT82-43](https://www.reichelt.com/be/en/ferrite-core-1-29-h-material-43-ami-ft-82-43-p245914.html): ⌀21mm, size core needed for 5W operation
    * [Fair-rite 5943001301](https://www.digikey.be/en/products/detail/fair-rite-products-corp/5943001301/8599633): ⌀25.4mm 
* [Unibalun](https://github.com/DG1JAN/UniBalun)

## Our own balun design
### Calculation : 
Given parameters:
* source impedance = 50ohm
* turns ratio = 1:7
* Max power = 5W
* Frequency range = 3-30MHz

In [23]:
import math
Zin = 50 # [ohm] source impedance
fmin = 3e6 # Lowest usable frequency[Hz]
turns_ratio = 7
Pmax = 5 # [W]

# Calculate impedance of primary coil
# Impedance of primary winding must be greater than 5x source impedance.
Xp_min = 5 * Zin

# Calculate inductance of primary coil
Lp_min = Xp_min / (2*math.pi*fmin)
print("Minimum required inductance is " + str(Lp_min) + "H")


Minimum required inductance is 1.3262911924324612e-05H


### Core materials
* [Fair-rite materials](https://www.fair-rite.com/materials/): Inductive Materials : 
 * [#43](https://www.fair-rite.com/43-material-data-sheet/) < 10MHz, permeability is very temperature dependent
 * [#61](https://www.fair-rite.com/61-material-data-sheet/) < 25MHz, much more stable permeability with respect to temperature.
 * [#67](https://www.fair-rite.com/67-material-data-sheet/) < 50MHz
 
#### Using a #43-core
This core is not suited for this frequency range according to the specs of the core material.  Let's try it anyway because it seems to common in the ham HF-world.
 
#### Using a #61-core
 * [Amidon FT 82-61  = Fair-rite 5961000601](https://www.digikey.be/en/products/detail/fair-rite-products-corp/5961000601/8594131) : ⌀21mm, #61, €1.69
 * [Fair-rite 5961001821](https://www.digikey.be/en/products/detail/fair-rite-products-corp/5961001821/15930539) : ⌀23mm, #61, €1.42, ⌀23mm
 
#### Using a #67-core
Let's select a 67-core because it matches the required frequency range: [FT87-67](https://www.amidoncorp.com/ft-87-67/). 

160 windings is too much to wind on such a small FT-67-87 core.  No core has been bought for this frequency range.

 ### Calculations

In [24]:
# Fair-Rite 5943001301 (#43 material)
#Al = 500 # [nH/N²] Inductance factor
#Ae = 0.308e-4 # [m²] Effective cross-sectional area
#Bmax = 0.22 # [T], from [Fair-rite materials](https://www.fair-rite.com/materials/): Inductive Materials

# Fair-Rite 5961001821 (#61 material)
Al = 75 # [nH/N²] Inductance factor
Ae = 0.262e-4 # [m²] Effictive cross-sectional area
Bmax = 0.25 # [T], from [Fair-rite materials](https://www.fair-rite.com/materials/): Inductive Materials

#Amidon FT-87-67 (#67 material)
#Al = 25 # [nH/N²] Inductance factor
#Ae = 0.2612e-4 # [m²] Effective cross-sectional area
#Bmax = 0.23 # [T], from [Fair-rite materials](https://www.fair-rite.com/materials/): Inductive Materials

# L = Al * N² (see also ARRL §5.7.3)
Np = math.sqrt(Lp_min/(Al*1e-9))
print("Required number of turns on primary winding: " + str(Np))
Ns = Np*turns_ratio
print("Required number of turns on secondary winding: " + str(Ns))

# Maximum coil voltage
Erms = math.sqrt( Pmax * Zin)

# Maximum flux density
B = Erms / (4.44 * Ae * Np * fmin) # [T]
print("Maximum flux density: " + str(B)+"T")

if B > Bmax:
    print("Exceeding maximum flux density")
else:
    print("Flux density OK")

Required number of turns on primary winding: 13.298076013381088
Required number of turns on secondary winding: 93.08653209366761
Maximum flux density: 0.0034070277986435003T
Flux density OK


### Winding the core 
* [Calculating needed wire (add 5cm)](https://mikeyancey.com/hamcalc/toroidcalculator.php)

For the 5961001821, the inner diameter is 12.9mm, so for about 100 windings, max. ⌀0.4mm wire can be used, to avoid windings to overlap.

### Measurements
Measurements are done using a NanoVNA showing the Smith chart.  The inductance value is shown on the left corner of its screen.

#### #43 material
Really not suited for the 5MHz to 30MHz range.  The inductance reduces drastically and losses rise fast.  Why is this material so popular?  Because only few windings are needed?

#### #61 material
Not suited for 5-30MHz.  For 13 windings, the inductance at 3MHz is 13.9µH while it even rises to 23µH at 30MHz.  This matches the µ'-curve of the material permeability versus frequency of the [#61 material datasheet](https://www.fair-rite.com/61-material-data-sheet/).


## Antenna calibration
* [Revised - Use of The NanoVNA For Antenna Measurements W5IFQ 7 AUG 2021](https://austinhams.org/wp-content/uploads/2021/08/Use-of-the-NanoVNA-for-Antenna-Measurements-AARC.pdf)

## References
* [R&S Understanding NVIS](https://www.youtube.com/watch?v=k3IFj_Z5cLM)
* [Local Emergency Communications with NVIS - Let's Build an 80m/40m Dipole](https://www.youtube.com/watch?v=yYXVKtu3nwk)
* ARRL Handbook 2018, §5.7.3 Broadband Ferrite RF Transformers
* Secrets of RF Circuit Design, Joseph J. Carr, §4. Winding your own coils
* [Antenna building #1 - EFHW Antenna - theory and practice of the broadband transformer : why capacitor needed on 49:1-impedance unun](https://www.youtube.com/watch?v=2-4J8ECkoe4)
* [U01 emergency communications Antenna - 80/60/40/20/17/15/10m EFHW](https://dl2man.de/u01-emergency-communications-antenna/)



1:1 Balun, e.g. Diamond BU-50

[Receiving WSPR on SDRplay RSPduo Shortwave Antenna Performance Test](https://www.youtube.com/watch?v=8lZds2MTKTI)